In [1]:
!apt-get update
!apt-get install -y ffmpeg

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2008 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:4 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5609 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [33.2 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [71.0 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3371 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]      
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1274 kB]
Get:12 http://arch

In [2]:
%pip install openai-whisper pandas datasets[audio]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 85.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 79.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 MB 101.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 88.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 101.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━

In [3]:
%pip install torchcodec --index-url=https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from datasets import load_dataset

dataset = load_dataset("cubbk/audio_swedish_2_dataset_cleaned",split='train', streaming=True, data_dir="8sidor_audios_dataset") 

print(next(iter(dataset)))
print(next(iter(dataset)))


{'.wav': <datasets.features._torchcodec.AudioDecoder object at 0x7fe411c32190>, '__key__': '000000', '__url__': 'hf://datasets/cubbk/audio_swedish_2_dataset_cleaned@e97329170fb6674f1f6bff0bf22626f46e4b1b98/8sidor_audios_dataset/dataset-000000.tar'}
{'.wav': <datasets.features._torchcodec.AudioDecoder object at 0x7fe411c32190>, '__key__': '000000', '__url__': 'hf://datasets/cubbk/audio_swedish_2_dataset_cleaned@e97329170fb6674f1f6bff0bf22626f46e4b1b98/8sidor_audios_dataset/dataset-000000.tar'}


In [5]:
# import whisper

# model = whisper.load_model("turbo")
# result = model.transcribe("https://8sidor.se/wp-content/uploads/2025/09/250919ny.mp3", language="Swedish", verbose=True)

# segments = result["segments"]

# segments_cleaned = []
# for segment in segments:
#     text = segment["text"].strip() # type: ignore
#     if text:
#         segments_cleaned.append({
#             "start": segment["start"], # type: ignore
#             "end": segment["end"], # type: ignore
#             "text": text,
#         })
    

# segments_cleaned

